## INIZIALIZATION

In [ ]:
import numpy as np
import time
from collections import namedtuple


solutions = np.empty(25, dtype=object)  # Increased size to handle more files
costs = np.empty(25, dtype=float)  

USE_GREEDY_SOLUTION=True
  
# Adaptive optimization parameters following 1-out-of-5 principle
μ0=50  # starting population size
k0=3   # starting offspring to parent ratio
λ0=k0*μ0 # starting offspring size
prx0=0.8  # starting recombination probability
pmx0=0.2  # starting mutation probability

MAX_GENERATIONS = 100  # maximum number of generations
ADAPTATION_INTERVAL = MAX_GENERATIONS // 10  # Update every 1/10 of total generations

# for exponential ranking
STARTING_P = 1.4
FINAL_P = 1.0
DECREASING_P = (STARTING_P - FINAL_P) / MAX_GENERATIONS

## Elaboration:

In [ ]:
# Review: change the individual definition as follows:
Individual = namedtuple('Individual', ['genotype', 'fitness'])
# By using such defined individuals you will be able to calculate fitness 
# only at the moment they are created instead of multiple times throughout the code

In [2]:
# Define cost function to be shared
def cost(solution, distance_matrix):
    total_cost = 0
    for i in range(len(solution) - 1):
        total_cost += distance_matrix[solution[i]][solution[i + 1]]
    return total_cost

def solve_tsp(distance_matrix): 
    
    class Link:
        def __init__(self, cost, source, destination):
            self.cost = cost
            self.source = source
            self.destination = destination
        
        def __lt__(self, other):
            return self.cost < other.cost

    # Check if solution is valid TSP tour: starts/ends with 0, visits all cities once
    def check_constraints(solution, n_cities):
        if solution[0] != 0 or solution[-1] != 0:
            return False
        if len(solution) != n_cities + 1:
            return False
        cities_visited = set(solution[1:-1])
        return len(cities_visited) == n_cities - 1
    
    def premature_cycle_detection(city_connections, link_to_add):
        city_connections_copy = city_connections.copy()
        city_connections_copy[link_to_add.source] = link_to_add.destination
        visited = set()
        current = link_to_add.source
        while current != -1:
            if current in visited:
                if len(visited) < len(city_connections_copy):    
                    return True
                else:  
                    return False
            visited.add(current)
            current = city_connections_copy[current]
        return False

    def greedy_starting_solution(distance_matrix, n_cities):
        # Create list of all links with their costs
        available_links = []
        for i in range(n_cities):
            for j in range(n_cities):
                if i != j:  
                    cost = distance_matrix[i][j]
                    available_links.append(Link(cost, i, j))
        
        available_links.sort()
        
        # Dictionary linking each city to its next destination (-1 means no connection yet)
        city_connections = {i: -1 for i in range(n_cities)} 
        used_links = 0
        
        # Process links in order of increasing cost
        i = 0
        while i < len(available_links) and used_links < n_cities:
            link = available_links[i]
            if premature_cycle_detection(city_connections, link):
                i += 1
                continue
            
            # Accept link if source has no outgoing connection
            if city_connections[link.source] == -1:
                city_connections[link.source] = link.destination
                used_links += 1
                
                # Remove conflicting links
                j = i + 1
                while j < len(available_links):
                    other_link = available_links[j]
                    if ((other_link.source == link.destination and other_link.destination == link.source) or  # Reverse link
                        (other_link.source == link.source) or  # Same source
                        (other_link.destination == link.destination)):  # Same destination
                        available_links.pop(j)
                    else:
                        j += 1
            i += 1
        
        # Build tour from connections starting at city 0
        tour = [0]
        current = 0
        
        while len(tour) <= n_cities:
            next_city = city_connections[current]
            tour.append(next_city)
            current = next_city
        
        return tour

    def random_solution(n_cities, main_rng):
        # Generate random valid tour
        cities = list(range(1, n_cities))
        main_rng.shuffle(cities)
        return [0] + cities + [0]

    def insert_mutation(solution, main_rng):
        # Insertion mutation: remove city and insert elsewhere
        new_solution = solution.copy()
        if len(new_solution) <= 3:  # Can't mutate tours with <= 2 cities
            return new_solution
            
        city_idx1 = main_rng.integers(1, len(new_solution) - 1)
        city_idx2 = main_rng.integers(1, len(new_solution) - 1)
        city = new_solution.pop(city_idx2)
        new_solution.insert(city_idx1+1, city)
        
        return new_solution

    # Review: it's seems more likely to be an order crossover (OX) than a cycle crossover
    def cycle_crossover(parent1, parent2):
        size = len(parent1) - 1  # Exclude the last 0
        offspring = [-1] * size
        offspring[0] = 0
        
        # Choose two random cut points (excluding first and last positions)
        if size > 2:
            cut1 = main_rng.integers(1, size - 1)
            cut2 = main_rng.integers(1, size - 1)
            if cut1 > cut2: cut1, cut2 = cut2, cut1
            
            # Ensure cut2 > cut1 for a valid segment
            if cut1 == cut2:
                cut2 = min(cut1 + 1, size - 1)
            
            for i in range(cut1, cut2):
                offspring[i] = parent1[i]
            
            # Get cities already used in the segment
            used_cities = set(offspring[cut1:cut2])
            
            # Fill remaining positions with cities from parent2 in order
            parent2_cities = [city for city in parent2[1:-1] if city not in used_cities]
            
            # Fill positions before cut1
            p2_idx = 0
            for i in range(1, cut1):
                if p2_idx < len(parent2_cities):
                    offspring[i] = parent2_cities[p2_idx]
                    p2_idx += 1
            
            # Fill positions after cut2
            for i in range(cut2, size):
                if p2_idx < len(parent2_cities):
                    offspring[i] = parent2_cities[p2_idx]
                    p2_idx += 1
        else:
            # If size <= 2, just copy parent1 (excluding endpoints)
            for i in range(1, size):
                offspring[i] = parent1[i]

        # Add final 0 and return
        return offspring + [0]

    def exponential_ranking_selection(population, fitness_scores, p, main_rng):
        n = len(population)
        # Make sure fitness_scores matches population size
        if len(fitness_scores) != n:
            fitness_scores = [-cost(sol, distance_matrix) for sol in population]
        
        indices = np.argsort(fitness_scores)  
        
        # Calculate selection probabilities
        probs = []
        for i in range(n):
            rank = n - indices[i] - 1  # Higher rank for better fitness
            prob = rank ** p
            probs.append(prob)
        
        # Normalize probabilities
        probs = np.array(probs)
        probs = probs / np.sum(probs)
        
        # Select individual
        selected_idx = main_rng.choice(len(population), p=probs)
        return population[selected_idx].copy()

    # Main EA loop with adaptive optimization
    n_cities = len(distance_matrix)
    main_rng = np.random.default_rng(seed=int(time.time() * 1000000000))
    
    # Initialize adaptive parameters
    μ = μ0
    k = k0
    λ = k * μ
    prx = prx0
    pmx = pmx0
    
    # Track individuals with fitness higher than previous best for 1/5 principle
    improvement_counts = []
    previous_best_fitness = float('-inf')  # Start with very low fitness
    
    # Initialize population
    population = []
    
    # Eventually add greedy solution
    if USE_GREEDY_SOLUTION:
        greedy_sol = greedy_starting_solution(distance_matrix, n_cities)
        if check_constraints(greedy_sol, n_cities):
            population.append(greedy_sol)
        # print("Greedy solution cost:", cost(greedy_sol, distance_matrix))

    # Fill rest with random solutions
    while len(population) < μ:
        sol = random_solution(n_cities, main_rng)
        if check_constraints(sol, n_cities):
            population.append(sol)
    
    current_p = STARTING_P
    
    for generation in range(MAX_GENERATIONS): 

        # fitness=-cost
        fitness_scores = [-cost(sol, distance_matrix) for sol in population]
        
        # Track individuals with fitness higher than previous best
        num_improvements = sum(1 for f in fitness_scores if f > previous_best_fitness)
        improvement_counts.append(num_improvements)
        
        # Update previous best fitness
        current_best_fitness = max(fitness_scores)
        if current_best_fitness > previous_best_fitness:
            previous_best_fitness = current_best_fitness
        
        # Adaptive parameter optimization every 1/10 of total generations
        if (generation + 1) % ADAPTATION_INTERVAL == 0 and generation > 0:
            # Calculate average number of individuals with fitness higher than previous best over this interval
            interval_start = max(0, generation + 1 - ADAPTATION_INTERVAL)
            avg_improvements = np.mean(improvement_counts[interval_start:generation + 1])
            
            # print(f"Generation {generation + 1}: Adaptive optimization check")
            # print(f"Average improvements: {avg_improvements:.2f}, Population size: {μ}")
            
            # Apply 1/5 principle: if avg < μ/5, increase selectivity, otherwise decrease
            if avg_improvements < μ / 5:
                # Increase selectivity: increase μ, k, prx; decrease pmx
                μ = int(μ * 1.1)
                k = k * 1.1
                λ = int(k * μ)
                prx = min(prx * 1.1, 0.95)  # Cap at 0.95
                pmx = max(pmx * 0.9, 0.05)  # Floor at 0.05
                # print(f"Increasing selectivity: μ={μ}, k={k:.2f}, λ={λ}, prx={prx:.3f}, pmx={pmx:.3f}")
            else:
                # Decrease selectivity: decrease μ, k, prx; increase pmx
                μ = max(int(μ * 0.9), 20)  # Floor at 20
                k = max(k * 0.9, 2.0)     # Floor at 2.0
                λ = int(k * μ)
                prx = max(prx * 0.9, 0.5) # Floor at 0.5
                pmx = min(pmx * 1.1, 0.4) # Cap at 0.4
                # print(f"Decreasing selectivity: μ={μ}, k={k:.2f}, λ={λ}, prx={prx:.3f}, pmx={pmx:.3f}")
            
            # Adjust population size if needed
            if len(population) > μ:
                # Keep only the best μ individuals
                current_fitness = [-cost(sol, distance_matrix) for sol in population]
                best_indices = np.argsort(current_fitness)[-μ:]
                population = [population[i] for i in best_indices]
            elif len(population) < μ:
                pass
                # Review: avoid to add random solutions during the evolution because it leads to a worst genetic pool quality
                # Add random individuals to reach μ
                #while len(population) < μ:
                #    sol = random_solution(n_cities, main_rng)
                #    if check_constraints(sol, n_cities):
                #        population.append(sol) 
        
        # Recalculate fitness scores after potential population changes
        fitness_scores = [-cost(sol, distance_matrix) for sol in population]
        
        offspring = []
        for _ in range(λ):
            if main_rng.random() < prx:  # Recombination
                parent1 = exponential_ranking_selection(population, fitness_scores, current_p, main_rng)
                parent2 = exponential_ranking_selection(population, fitness_scores, current_p, main_rng)
                child = cycle_crossover(parent1, parent2)
                if check_constraints(child, n_cities):
                    offspring.append(child)
            else:  # Just selection
                parent = exponential_ranking_selection(population, fitness_scores, current_p, main_rng)
                offspring.append(parent.copy())
        
        # Mutation
        for i in range(len(offspring)):
            if main_rng.random() < pmx:
                candidate = insert_mutation(offspring[i], main_rng)
                if check_constraints(candidate, n_cities):
                    offspring[i] = candidate

        # Survival selection (μ + λ)
        combined = population + offspring
        combined_fitness = [-cost(sol, distance_matrix) for sol in combined]
        
        # Select best μ individuals
        best_indices = np.argsort(combined_fitness)[-μ:]
        population = [combined[i] for i in best_indices]
        
        # Decrease p
        current_p = max(FINAL_P, current_p - DECREASING_P)
    
    # Return best solution
    final_fitness = [-cost(sol, distance_matrix) for sol in population]
    best_idx = np.argmax(final_fitness)
    return population[best_idx]

In [3]:
import glob

# Get all .npy files in the problems folder
problem_files = glob.glob("problems/*.npy")
problem_files.sort()  

i = 0
for problem_file in problem_files:
 
    print(f"Processing {problem_file}...")
    
    distance_matrix = np.load(problem_file)
    solution = solve_tsp(distance_matrix)
    print(f"Solution: {solution}")
    
    solutions[i] = solution
    costs[i] = cost(solution, distance_matrix)

    print(f" Cost: {round(costs[i])}")
    print("-" * 50)
    i += 1

Processing problems\problem_g_10.npy...
Solution: [0, 7, 9, 5, 4, 6, 1, 3, 2, 8, 0]
 Cost: 1498
--------------------------------------------------
Processing problems\problem_g_100.npy...
Solution: [0, 33, 54, 67, 32, 37, 5, 49, 7, 99, 20, 27, 74, 71, 47, 63, 84, 3, 51, 28, 21, 83, 89, 85, 69, 22, 1, 8, 36, 46, 4, 34, 75, 93, 55, 86, 88, 78, 92, 43, 87, 6, 77, 50, 15, 91, 12, 24, 40, 19, 80, 25, 26, 13, 76, 65, 39, 98, 96, 29, 23, 61, 9, 59, 82, 62, 90, 38, 95, 35, 17, 44, 94, 79, 66, 45, 73, 53, 16, 52, 68, 81, 30, 97, 64, 42, 10, 48, 31, 72, 18, 56, 70, 58, 2, 11, 60, 57, 14, 41, 0]
 Cost: 4853
--------------------------------------------------
Processing problems\problem_g_1000.npy...
Solution: [0, 951, 379, 950, 77, 768, 483, 773, 134, 760, 361, 878, 284, 32, 172, 692, 292, 720, 375, 886, 155, 57, 636, 752, 470, 352, 546, 716, 674, 437, 119, 543, 127, 779, 499, 269, 570, 491, 928, 48, 145, 654, 853, 881, 762, 186, 429, 459, 75, 376, 593, 71, 390, 834, 250, 890, 427, 192, 597, 617, 